In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import json
import time
from dotenv import load_dotenv

In [2]:
user_train_path = '../train_valid_test/user_train.pkl'
user_valid_path = '../train_valid_test/user_valid.pkl'
user_test_path = '../train_valid_test/user_test.pkl'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = pickle.load(f)
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)

In [4]:
user_id = 1
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])
print(len(user_train_dict[user_id][0]))

[[1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907]]
[1566]
[48]
36


In [5]:
user_profile_path = '../user_profile/cleaned_user_profile_mixtral.pkl'
with open(user_profile_path, 'rb') as f:
    user_profile_dict = pickle.load(f)
print(user_profile_dict[user_id])


            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.
            
            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.
            
            The user's taste in movies also extends to comedies, as seen in their appreciation for the hilarious antics of Wallace & Gromit and the dark humor of Fargo. They are also drawn to adventure films, as demonstrated by their love for Star Wars: Episode IV - A New Hope and Back to the Future.
            
            However, the user is not a fan of 

In [6]:
movie_summary_df = pd.read_csv('../movie_summary_25_words.csv')
movie_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [7]:
print(movie_summary_df[movie_summary_df['movie_id'] == 265])

     movie_id                                               name  \
262       265  Like Water for Chocolate (Como agua para choco...   

     cleaned_genre  year                                            summary  
262  Drama,Romance  1992  "Like Water for Chocolate is a visually stunni...  


In [8]:
user_df = pd.read_csv('../users.csv')
user_df.head()

,user_id,user_profile
0,1,female k-12 student under 18 years of age
1,2,male self-employed above 56 years of age
2,3,male scientist between 25 to 34 years of age
3,4,male executive/managerial between 45 to 49 yea...
4,5,male writer between 25 to 34 years of age


In [9]:
ratings_df = pd.read_csv('../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [10]:
# reasoning_prompt_train_data = "./reasoning_dataset/reasoning_prompt_data/reasoning_prompt_train_new.pkl"
reasoning_train_data = "./reasoning_dataset/reasoning_data/reasoning_train_dict.pkl"
reasoning_valid_data = "./reasoning_dataset/reasoning_data/reasoning_valid_dict.pkl"
reasoning_test_data = "./reasoning_dataset/reasoning_data/reasoning_test_dict.pkl"

In [11]:
with open(reasoning_train_data, 'rb') as f:
    reasoning_train = pickle.load(f)
with open(reasoning_valid_data, 'rb') as f:
    reasoning_valid = pickle.load(f)
with open(reasoning_test_data, 'rb') as f:
    reasoning_test = pickle.load(f)

In [12]:
# reasoning_train_data = "./reasoning_dataset/reasoning_data_wo_profile/reasoning_train_dict.pkl"
# reasoning_valid_data = "./reasoning_dataset/reasoning_data_wo_profile/reasoning_valid_dict.pkl"
# reasoning_test_data = "./reasoning_dataset/reasoning_data_wo_profile/reasoning_test_dict.pkl"

In [13]:
# with open(reasoning_train_data, 'rb') as f:
#     reasoning_train = pickle.load(f)
# with open(reasoning_valid_data, 'rb') as f:
#     reasoning_valid = pickle.load(f)
# with open(reasoning_test_data, 'rb') as f:
#     reasoning_test = pickle.load(f)

In [12]:
# print(reasoning_prompt_train[user_id])
print(reasoning_train[user_id])
print(reasoning_valid[user_id])
print(reasoning_test[user_id])


Summary of user behavior and movie characteristics:
The user enjoys a diverse range of movies, with a preference for thought-provoking and emotionally engaging films. They appreciate enchanting storytelling, timeless charm, and humor. The user is drawn to movies that challenge their perspective, explore complex themes, and provoke deep thought. They enjoy dramas, classic Disney films, comedies, and adventure films, but avoid horror and overly violent or graphic movies.

Reasoning for why the user will like Mulan:
Mulan is a captivating animated film that tells an empowering story of a young Chinese woman who disguises herself as a man to fight in the army. This movie aligns with the user's preference for thought-provoking and emotionally engaging films, as it explores themes of identity, courage, and self-discovery. The user's appreciation for classic Disney films and enchanting storytelling is also satisfied by Mulan's stunning animation and unforgettable music. Additionally, the mov

In [13]:
def get_new_reasoning_text(reasoning):
    # print(f"reasoning: {reasoning}")
    new_reasoning_text = f"Prediction: {reasoning.split(' ')[-1]} \n"
    new_reasoning_text += ' '.join(s.strip() for s in reasoning.split("\n")[:-1])
    # print(f"new_reasoning_text: {new_reasoning_text}")
    return new_reasoning_text

In [14]:
final_train_dict = list()
cnt = 0
max_total_len = 0
avg_total_len = 0
for user, user_profile in user_profile_dict.items():
    if user not in reasoning_test:
        continue
    # if cnt < 1999:
    #     cnt += 1
    #     continue
    cnt += 1
    temp_dict = dict()
    # print(f"User: {user}")
    temp_dict["instruction"] = "You are an expert movie critic and recommender. Given the user profile and list of recent liked and disliked movies, identify whether the user will like the target movie or not by providing reasoning for the same. Please follow further instruction below."
    prompt = "User Profile - "
    ### Create the user profile
    # final_user_profile = "The user is a "+user_df[user_df['user_id'] == user]['user_profile'].values[0]+user_profile+"\n"
    # print(final_user_profile)
    prompt += user_profile
    
    prompt += "\nUser watched the following movies recently in the given order. List of user behaviour for recent movies and their description -"
    # print(prompt)
    ### Create the like/dislike list of last 20 movies
    # print(user_train_dict[user][0][-20:])
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    liked_movies = []
    disliked_movies = []

    train_valid_movie_list = user_train_dict[user][0][-19:]
    train_valid_movie_list.append(user_valid_dict[user][0])
    target_movie = user_test_dict[user][0]

    for movie in train_valid_movie_list:
        user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
        # print(movie, user_movie_rating)
        movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
        if user_movie_rating >= 3:
            if movie != target_movie:
                prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " + movie_details['summary'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"
        else:
            if movie != target_movie:
                prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"

    ### To predict
    prompt += "\nThe user watched the next target movie - "
    # print(user_valid_dict[user][0])
    target_movie_details = movie_summary_df[movie_summary_df['movie_id'] == target_movie]
    # print("Test Movie Rating:", user_rating_dict[user_rating_dict['movie_id'] == user_valid_dict[user][0]]['rating'].values[0])
    prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
    prompt += f"""
    Do not use any information not mentioned above. As an expert movie critic and recommender, do the following steps -
    Analyze the user profile and movie watching history given in order and summarize the user's behavior from the given list of movies and identify the movie characteristics that he likes and dislikes in atmost 100 words.
    Explain with reasoning whether the user will like or dislike the target movie - {target_movie_details['name'].values[0]} in atmost 100 words.
    Predict whether the user will like the target movie or not in the following format - Prediction: Yes or No
    """
    temp_dict["input"] = prompt
    '''
    Found the issue that when the Prediction: Yes is at the very end, during training the output, as is parallelized, always performs good as it already has the information till the n-1th token, so the AUC saturates in the first epoch itself. Hence changing the position of 'Prediction: Yes' to starting
    '''
    temp_dict["output"] = get_new_reasoning_text(reasoning_test[user])

    final_train_dict.append(temp_dict)
    total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
    avg_total_len += total_len
    max_total_len = max(max_total_len, total_len)
    # print(f"Prompt length: {len(prompt.split())}")
    # print(f"Total length: {len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())}")
    # print(final_train_dict)
    # break

print("Length of final_train_dict:", len(final_train_dict), final_train_dict[0])
print("Avg Length:", avg_total_len/len(final_train_dict))
print("Max length:", max_total_len)

Length of final_train_dict: 6040 {'instruction': 'You are an expert movie critic and recommender. Given the user profile and list of recent liked and disliked movies, identify whether the user will like the target movie or not by providing reasoning for the same. Please follow further instruction below.', 'input': 'User Profile - \n            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.\n            \n            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.\n            \n            The user\'s 

In [16]:
# final_train_dict = list()
# cnt = 0
# max_total_len = 0
# avg_total_len = 0
# for user, user_profile in user_profile_dict.items():
#     if user not in reasoning_train:
#         continue
#     # if cnt < 1999:
#     #     cnt += 1
#     #     continue
#     cnt += 1
#     temp_dict = dict()
#     # print(f"User: {user}")
#     prompt = "\nUser watched the following movies recently in the given order. List of user behaviour for recent movies and their description -"
#     temp_dict["instruction"] = "You are an expert movie critic and recommender. Given the list of recent liked and disliked movies and their description, identify whether the user will like the target movie or not by providing reasoning for the same. Please follow further instruction below."
#     # print(prompt)
#     ### Create the like/dislike list of last 20 movies
#     # print(user_train_dict[user][0][-20:])
#     user_rating_dict = ratings_df[ratings_df['user_id'] == user]
#     # print(user_rating_dict)
#     liked_movies = []
#     disliked_movies = []

#     train_valid_movie_list = user_train_dict[user][0][-21:-1]
#     # train_valid_movie_list.append(user_valid_dict[user][0])
#     target_movie = user_train_dict[user][0][-1]

#     for movie in train_valid_movie_list:
#         user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
#         # print(movie, user_movie_rating)
#         movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
#         if user_movie_rating >= 3:
#             if movie != target_movie:
#                 prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " + movie_details['summary'].values[0] + ","
#             else:
#                 ### For the last movie - ending with a . and starting new line
#                 prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"
#         else:
#             if movie != target_movie:
#                 prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + ","
#             else:
#                 ### For the last movie - ending with a . and starting new line
#                 prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"

#     ### To predict
#     prompt += "\nThe user watched the next target movie - "
#     # print(user_valid_dict[user][0])
#     target_movie_details = movie_summary_df[movie_summary_df['movie_id'] == target_movie]
#     # print("Test Movie Rating:", user_rating_dict[user_rating_dict['movie_id'] == user_valid_dict[user][0]]['rating'].values[0])
#     prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
#     prompt += f"""\nDo not use any information not mentioned above. As an expert movie critic and recommender, do the following steps -
#     Analyze the user's movie watching history given in order and summarize the user's behavior from the given list of movies and identify the movie characteristics that he likes and dislikes in atmost 100 words.
#     Explain with reasoning whether the user will like or dislike the target movie - {target_movie_details['name'].values[0]} in atmost 100 words.
#     Finally, Predict whether the user will like the target movie or not in the following format - Prediction: Yes or No"""
#     temp_dict["input"] = prompt
#     temp_dict["output"] = reasoning_train[user]
#     final_train_dict.append(temp_dict)
#     total_len = len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())
#     avg_total_len += total_len
#     max_total_len = max(max_total_len, total_len)
#     # print(f"Prompt length: {len(prompt.split())}")
#     # print(f"Total length: {len(temp_dict['instruction'].split()) + len(temp_dict['input'].split()) + len(temp_dict['output'].split())}")
#     # print(final_train_dict)
#     # break

# print("Length of final_train_dict:", len(final_train_dict), final_train_dict[0])
# print("Avg Length:", avg_total_len/len(final_train_dict))
# print("Max length:", max_total_len)

Length of final_train_dict: 4800 {'instruction': 'You are an expert movie critic and recommender. Given the list of recent liked and disliked movies and their description, identify whether the user will like the target movie or not by providing reasoning for the same. Please follow further instruction below.', 'input': '\nUser watched the following movies recently in the given order. List of user behaviour for recent movies and their description -\nLiked James and the Giant Peach (Genre: Animation,Children\'s,Musical) "James and the Giant Peach (1996) is a whimsical and visually stunning adaptation of Roald Dahl\'s beloved children\'s book, filled with adventure, friendship, and magic.",\nLiked Ferris Bueller\'s Day Off (Genre: Comedy) "Ferris Bueller\'s Day Off" is a classic 1980s comedy about a high school student who fakes being sick to have an epic day of adventure and mischief in Chicago.,\nLiked Secret Garden, The (Genre: Children\'s,Drama) "Secret Garden (1993) is a heartwarming

In [15]:
final_train_dict[1]

{'instruction': 'You are an expert movie critic and recommender. Given the user profile and list of recent liked and disliked movies, identify whether the user will like the target movie or not by providing reasoning for the same. Please follow further instruction below.',
 'input': 'User Profile - \n            The user has a diverse taste in movies, with a preference for classic films and comedies. They enjoy movies that are both visually stunning and emotionally impactful, such as Fantasia and Children of a Lesser God. The user is drawn to movies that offer a unique perspective, like Anna and the King, and is not afraid to explore darker themes, as evidenced by their appreciation for American Beauty and The Black Cauldron.\n            They have a soft spot for romantic comedies, as seen in their fondness for Pretty Woman, and are also a fan of action-packed films like Jurassic Park and its sequel, The Lost World: Jurassic Park. The user is not limited to a specific genre, as they a

In [16]:
yes_cnt = 0
no_cnt = 0
for train_data in final_train_dict:
    pred = train_data['output'].split(' ')[1]
    # print(pred)
    if pred == 'Yes':
        yes_cnt += 1
    elif pred == 'No':
        no_cnt += 1
    else:
        print(pred, '--', train_data['output'])
print(yes_cnt, no_cnt, yes_cnt + no_cnt)

4974 1066 6040


In [17]:
final_train_path = "./final_data/movie_new/test.json"
with open(final_train_path, 'w') as f:
    json.dump(final_train_dict, f)